In [1]:
from transformers import BertForSequenceClassification, BertTokenizer
import pandas as pd
import numpy as np
import torch
from torch.utils.data import DataLoader, TensorDataset
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [3]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

In [4]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [5]:
model.classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 1),
    nn.Sigmoid()
)

In [6]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [7]:
df = pd.read_csv("/content/fakenews.csv")
df.head()

,text,label
0,Get the latest from TODAY Sign up for our news...,1
1,2d Conan On The Funeral Trump Will Be Invited...,1
2,It’s safe to say that Instagram Stories has fa...,0
3,Much like a certain Amazon goddess with a lass...,0
4,At a time when the perfect outfit is just one ...,0


In [8]:
sentences_train, sentences_test, labels_train, labels_test = train_test_split(list(df["text"]), list(df["label"]), test_size = 0.20)

In [9]:
tokenized_train = tokenizer(sentences_train, padding=True, truncation=True, return_tensors='pt')
tokenized_test = tokenizer(sentences_test, padding=True, truncation=True, return_tensors='pt')

labels_train = torch.tensor(labels_train)
labels_test = torch.tensor(labels_test)

In [10]:
dataset_train = TensorDataset(tokenized_train['input_ids'], tokenized_train['attention_mask'], labels_train)
dataset_test = TensorDataset(tokenized_test['input_ids'], tokenized_test['attention_mask'], labels_test)

train_loader = DataLoader(dataset_train, batch_size=8, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size=8, shuffle=True)

## Training only last classification layer of BERT model

In [11]:
trainable_layer_names = ["classifier"]

In [12]:
for name, module in model.named_children():
    if name in trainable_layer_names:
        for param in module.parameters():
            param.requires_grad = True
    else:
        for param in module.parameters():
            param.requires_grad = False

In [13]:
for name, param in model.named_parameters():
    print(f'{name}: requires_grad={param.requires_grad}')

bert.embeddings.word_embeddings.weight: requires_grad=False
bert.embeddings.position_embeddings.weight: requires_grad=False
bert.embeddings.token_type_embeddings.weight: requires_grad=False
bert.embeddings.LayerNorm.weight: requires_grad=False
bert.embeddings.LayerNorm.bias: requires_grad=False
bert.encoder.layer.0.attention.self.query.weight: requires_grad=False
bert.encoder.layer.0.attention.self.query.bias: requires_grad=False
bert.encoder.layer.0.attention.self.key.weight: requires_grad=False
bert.encoder.layer.0.attention.self.key.bias: requires_grad=False
bert.encoder.layer.0.attention.self.value.weight: requires_grad=False
bert.encoder.layer.0.attention.self.value.bias: requires_grad=False
bert.encoder.layer.0.attention.output.dense.weight: requires_grad=False
bert.encoder.layer.0.attention.output.dense.bias: requires_grad=False
bert.encoder.layer.0.attention.output.LayerNorm.weight: requires_grad=False
bert.encoder.layer.0.attention.output.LayerNorm.bias: requires_grad=False
be

In [20]:
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()

In [16]:
model = model.to(device)

In [21]:
model.train()

num_epochs = 3
for epoch in range(num_epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

    optimizer.zero_grad()

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
    if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print("Finished Training")

[1,   100] loss: 0.658
[1,   200] loss: 0.654
[1,   300] loss: 0.667
[1,   400] loss: 0.665
[2,   100] loss: 0.668
[2,   200] loss: 0.665
[2,   300] loss: 0.672
[2,   400] loss: 0.660
[3,   100] loss: 0.658
[3,   200] loss: 0.678
[3,   300] loss: 0.650
[3,   400] loss: 0.649
Finished Training


In [22]:
model.eval()

val_loss = 0
with torch.no_grad():
  for data in test_loader:
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    val_loss += loss.item()
print(f"Loss on test data: {val_loss / len(test_loader)}")

Loss on test data: 0.6553111584186554


In [23]:
total_samples = 0
correct_predictions = 0

model.eval()

with torch.no_grad():
    for data in test_loader:
        input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.round(logits)

        total_samples += labels.size(0)

        correct_predictions += (predictions.cpu().numpy().T==labels.cpu().numpy()).sum().item()

accuracy = correct_predictions / total_samples
print(f"Accuracy on test data: {accuracy:.2%}")

Accuracy on test data: 59.92%


In [24]:
model.save_pretrained("fine_tuned_bert")

## Adding a new layer to the BERT model

In [25]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [26]:
num_classes = 1
classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
    nn.Sigmoid()
)
model.classifier = classifier

In [27]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [28]:
trainable_layer_names = ["classifier"]

In [29]:
for name, module in model.named_children():
    if name in trainable_layer_names:
        for param in module.parameters():
            param.requires_grad = True
    else:
        for param in module.parameters():
            param.requires_grad = False

In [30]:
for name, param in model.named_parameters():
    print(f'{name}: requires_grad={param.requires_grad}')

bert.embeddings.word_embeddings.weight: requires_grad=False
bert.embeddings.position_embeddings.weight: requires_grad=False
bert.embeddings.token_type_embeddings.weight: requires_grad=False
bert.embeddings.LayerNorm.weight: requires_grad=False
bert.embeddings.LayerNorm.bias: requires_grad=False
bert.encoder.layer.0.attention.self.query.weight: requires_grad=False
bert.encoder.layer.0.attention.self.query.bias: requires_grad=False
bert.encoder.layer.0.attention.self.key.weight: requires_grad=False
bert.encoder.layer.0.attention.self.key.bias: requires_grad=False
bert.encoder.layer.0.attention.self.value.weight: requires_grad=False
bert.encoder.layer.0.attention.self.value.bias: requires_grad=False
bert.encoder.layer.0.attention.output.dense.weight: requires_grad=False
bert.encoder.layer.0.attention.output.dense.bias: requires_grad=False
bert.encoder.layer.0.attention.output.LayerNorm.weight: requires_grad=False
bert.encoder.layer.0.attention.output.LayerNorm.bias: requires_grad=False
be

In [31]:
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()

In [32]:
model = model.to(device)

In [35]:
model.train()

num_epochs = 3
for epoch in range(num_epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

    optimizer.zero_grad()

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
    if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print("Finished Training")

[1,   100] loss: 0.670
[1,   200] loss: 0.678
[1,   300] loss: 0.646
[1,   400] loss: 0.664
[2,   100] loss: 0.647
[2,   200] loss: 0.663
[2,   300] loss: 0.660
[2,   400] loss: 0.676
[3,   100] loss: 0.663
[3,   200] loss: 0.644
[3,   300] loss: 0.646
[3,   400] loss: 0.669
Finished Training


In [36]:
model.eval()

val_loss = 0
with torch.no_grad():
  for data in test_loader:
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    val_loss += loss.item()
print(f"Loss on test data: {val_loss / len(test_loader)}")

Loss on test data: 0.6547737016677856


In [37]:
total_samples = 0
correct_predictions = 0

model.eval()

with torch.no_grad():
    for data in test_loader:
        input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.round(logits)

        total_samples += labels.size(0)

        correct_predictions += (predictions.cpu().numpy().T==labels.cpu().numpy()).sum().item()

accuracy = correct_predictions / total_samples
print(f"Accuracy on test data: {accuracy:.2%}")

Accuracy on test data: 63.93%


In [38]:
model.save_pretrained("fine_tuned_bert_extra_layer_added")

## Training the entire BERT model

In [39]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=1)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [40]:
num_classes = 1
classifier = nn.Sequential(
    nn.Linear(model.config.hidden_size, 256),
    nn.ReLU(),
    nn.Linear(256, num_classes),
    nn.Sigmoid()
)
model.classifier = classifier

In [41]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [42]:
optimizer = optim.AdamW(model.parameters(), lr=2e-5)
criterion = nn.BCELoss()

In [43]:
model = model.to(device)

In [44]:
model.train()

num_epochs = 3
for epoch in range(num_epochs):
  running_loss = 0.0
  for i, data in enumerate(train_loader, 0):
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

    optimizer.zero_grad()

    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    running_loss += loss.item()
    loss.backward()
    optimizer.step()
    if i % 100 == 99:
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0

print("Finished Training")

[1,   100] loss: 0.656
[1,   200] loss: 0.640
[1,   300] loss: 0.593
[1,   400] loss: 0.523
[2,   100] loss: 0.413
[2,   200] loss: 0.404
[2,   300] loss: 0.435
[2,   400] loss: 0.373
[3,   100] loss: 0.255
[3,   200] loss: 0.281
[3,   300] loss: 0.293
[3,   400] loss: 0.272
Finished Training


In [45]:
model.eval()

val_loss = 0
with torch.no_grad():
  for data in test_loader:
    input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)
    outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
    logits = outputs.logits

    loss = criterion(logits.squeeze(), labels.float())
    val_loss += loss.item()
print(f"Loss on test data: {val_loss / len(test_loader)}")

Loss on test data: 0.4604013759493828


In [46]:
total_samples = 0
correct_predictions = 0

model.eval()

with torch.no_grad():
    for data in test_loader:
        input_ids, attention_mask, labels = data[0].to(device), data[1].to(device), data[2].to(device)

        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        predictions = torch.round(logits)

        total_samples += labels.size(0)

        correct_predictions += (predictions.cpu().numpy().T==labels.cpu().numpy()).sum().item()

accuracy = correct_predictions / total_samples
print(f"Accuracy on test data: {accuracy:.2%}")

Accuracy on test data: 80.66%


In [47]:
model.save_pretrained("fine_tuned_bert_fully_trained")